In [ ]:
#Jupyter Notebook settings
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.image as mpimg
import scipy as sp
import matplotlib.pyplot as plt
from lxml import etree
from litdrive.roads.road_list import *
from litdrive.roads.LITD_RoadManager import *
from litdrive.roads.road_access import *

In [ ]:
picture_path="../../../configuration_files/maps/aadc2018#test#track#003_litd.jpg"
pickle_path="../../../configuration_files/maps/qualifying_2018_litd.pickle"

In [ ]:
rl = RoadList()
rl.loadFromFile(pickle_path)

In [ ]:
#Plot the track on the picture of the map.

%matplotlib notebook

#pixel per meter
pic_ppm=100.0
pic_offset_x=0.975
pic_offset_y=0.975
img_mat=mpimg.imread(picture_path)
img_mat=np.flipud(img_mat)

plt.figure(figsize=(28.5,14))
plt.axis([0, 30*pic_ppm, 0, 16*pic_ppm])

plt.imshow(img_mat)

#settings to view a junction
#plt.axis([0, 2, 4.5, 6.5])

#plt.axis([8.5, 10.5, 14.5, 16.5])

roads_len=0.0

#for key,l in rl.roads.items():
#    if(l is not None):
#        print(l)

for key,l in rl.lanes.items():
    if(l is not None):
        pts=l.getPixelPointList(pic_ppm,5)
        roads_len+=l.calcArcLength()
        x=pts[0]+pic_offset_x*pic_ppm
        y=pts[1]+pic_offset_y*pic_ppm
        plt.plot(x,y, '-')
        #plt.quiver(x[:-1], y[:-1], x[1:]-x[:-1], y[1:]-y[:-1], scale_units='xy', angles='xy', scale=1)
        
        
        if(True):
        
        #if(l.is_junction):
        #if(key==60 or key==178 or key==64):
        
        #for the merge lane
        #if(l.road_id==36 or l.road_id==110 or l.road_id==158 or l.road_id==171 or l.road_id==172 or l.road_id==173):
            #plt.text(x[len(x)//2],y[len(y)//2],str(key), color="blue", size="large")
            p1d_dx, p1d_dy= l.calcPolyDerivate()
            dy=p1d_dy(0.5)
            dx=p1d_dx(0.5)
            angle=np.arctan2(dy,dx)-np.pi/2.0
            dx=0.6*np.cos(angle)
            dy=0.6*np.sin(angle)
            plt.annotate(str(key), xy=(x[len(x)//2], y[len(y)//2]), xytext=(x[len(x)//2]+dx*pic_ppm, y[len(y)//2]+dy*pic_ppm),  arrowprops=dict(facecolor='black', shrink=0.05), )
        #for i in range(0,len(pts)-1):
        #    plt.arrow(pts[0][i],pts[1][i],pts[0][i+1]-pts[0][i],pts[1][i+1]-pts[1][i],,head_width=0.02, head_length=0.02, fc='black', ec='black')

print("Length is {}".format(roads_len))
#plt.text(xs[1]+0.2,ys[1]+0.2,s)#+'  '+str(degs[3]*180/np.pi))
#plt.plot(xs,ys,'g.')



In [ ]:
#Check, that the lane successors and predecessors are not further appart then 4cm

print("Checking {} elements, against {} successors and {} predecessors.".format(len(rl.lanes), len(rl.successors), len(rl.predecessors)))
for key,l in rl.lanes.items():
    if(l is not None):
        x_1=l.x_poly(1.0)
        y_1=l.y_poly(1.0)
        x_0=l.x_poly(0.0)
        y_0=l.y_poly(0.0)
        suc_dict=rl.successors[key]
        if(suc_dict is not None):
            for key_2, suc_id in suc_dict.items():
                s=rl.lanes[suc_id]
                xs_0=s.x_poly(0.0)
                ys_0=s.y_poly(0.0)
                dx=abs(x_1-xs_0)
                dy=abs(y_1-ys_0)
                delta=np.sqrt(dx**2+dy**2)
                if(delta>0.04):
                    print("ERROR: Difference between lane {} and successor {} is dx={} and dy={}!".format(key,suc_id,dx,dy))
        else:
            print("WARNING: lane {} (road {}) has no successor!".format(key, l.road_id))
        pre_dict=rl.predecessors[key]
        if(pre_dict is not None):
            for key_2, pre_id in pre_dict.items():
                p=rl.lanes[pre_id]
                xs_1=p.x_poly(1.0)
                ys_1=p.y_poly(1.0)
                dx=abs(x_0-xs_1)
                dy=abs(y_0-ys_1)
                delta=np.sqrt(dx**2+dy**2)
                if(delta>0.04):
                    print("ERROR: Difference between lane {} and predecessor {} is dx={} and dy={}!".format(key,pre_id,dx,dy))
        else:
            print("WARNING: lane {} (road {}) has no predecessor!".format(key, l.road_id))
            
print("Done!")

In [ ]:
lst=getLaneListByDecisions(rl, 201, [RoadDecisions.LEFT, RoadDecisions.RIGHT, RoadDecisions.STRAIGHT, RoadDecisions.RIGHT, RoadDecisions.LEFT], 20)

In [ ]:
print(lst)